In [5]:
def str_to_bin(str):
    bin = ""
    for c in str:
        bin += format(ord(c), "b").zfill(8)
    return bin


def bin_to_int(bin):
    return int(bin, 2)


def bin_to_str(bin):
    str = ""
    for i in range(0, len(bin), 8):
        str += chr(int(bin[i : i + 8], 2))
    return str

In [ ]:
class RadixTree:
    """
    Uma árvore de prefixos compacta.
    """

    def __init__(self) -> None:
        """
        Cria uma árvore de prefixos compacta vazia.
        >>> RadixTree()
        """
        # O nó raiz da árvore
        self.root = RadixNode()
        # O número de palavras na árvore
        self.size = 0
        # Próximo índice
        self.next_index = 0

    def insert(self, word: str) -> None:
        """
        Insere uma palavra na árvore.

        Args:
            word (str): palavra a ser inserida.

        >>> RadixTree().insert("word")
        """
        index = self.next_index
        self.root.insert(word, index)
        self.next_index += 1
        self.size += 1

    def insert_many(self, words: list[str]) -> None:
        """
        Insere várias palavras na árvore.

        Args:
            words (list[str]): lista de palavras a serem inseridas.

        >>> RadixTree().insert_many(["word1", "word2"])
        """
        for word in words:
            self.insert(word)

    def find(self, word: str) -> bool:
        """
        Verifica se uma palavra está na árvore.

        Args:
            word (str): palavra a ser verificada.

        Returns:
            bool: True se a palavra estiver na árvore,
                  False caso contrário.

        >>> RadixTree().insert("word")
        >>> RadixTree().find("word")
        True
        """
        return self.root.find(word)

    def delete(self, word: str) -> bool:
        """
        Deleta uma palavra da árvore.

        Args:
            word (str): palavra a ser deletada.

        Returns:
            bool: True caso a palavra esteja na árvore e for deletada,
                  False caso a palavra não esteja na árvore.

        >>> RadixTree().insert("word")
        >>> RadixTree().delete("word")
        True
        """
        if self.root.delete(word):
            self.size -= 1
            return True
        return False

    def print_tree(self) -> None:
        """
        Imprime a árvore.

        >>> RadixTree().insert_many(["word1", "word2"])
        >>> RadixTree().print_tree()
        --- word
        ------ 1:   (0)
        ------ 2:   (1)
        """
        self.root.print_tree()

    def all_words(self) -> list[(str, int)]:
        """
        Retorna todas as palavras da árvore numa lista.

        Returns:
            list[(str, int)]: lista de palavras e índices.

        >>> RadixTree().insert_many(["word1", "word2"])
        >>> RadixTree().all_words()
        [('word1', 0), ('word2', 1)]
        """
        return self.root.all_words()

    def __repr__(self) -> str:
        """
        >>> RadixTree().insert_many(["word1", "word2"])
        >>> RadixTree()
        [('word1', 0), ('word2', 1)]
        """
        return str(self.all_words())

    def __contains__(self, word: str) -> bool:
        """
        >>> RadixTree().insert("word")
        >>> "word" in RadixTree()
        True
        """
        return self.find(word)

    def __iter__(self):
        """
        >>> RadixTree().insert_many(["word1", "word2"])
        >>> [(word, i) for (word, i) in RadixTree()]
        [('word1', 0), ('word2', 1)]
        """
        return iter(self.all_words())

    def __add__(self, word: str) -> None:
        """
        >>> RadixTree() + "word"
        >>> "word" in RadixTree()
        True
        """
        self.insert(word)

    def __iadd__(self, word: str):
        """
        >>> RadixTree() += "word"
        >>> "word" in RadixTree()
        True
        """
        self.insert(word)
        return self

    def __sub__(self, word: str) -> bool:
        """
        >>> RadixTree().insert("word")
        >>> RadixTree() - "word"
        >>> "word" not in RadixTree()
        True
        """
        return self.delete(word)

    def __isub__(self, word: str):
        """
        >>> RadixTree().insert("word")
        >>> RadixTree() -= "word"
        >>> "word" not in RadixTree()
        True
        """
        self.delete(word)
        return self

    def __len__(self) -> int:
        """
        >>> RadixTree().insert_many(["word1", "word2"])
        >>> len(RadixTree())
        2
        """
        return self.size

    def sort(self) -> list[(str, int)]:
        """
        >>> RadixTree().insert_many(["word0", "word1"])
        >>> RadixTree().sort()
        [('word0', 0), ('word1', 1)]
        """
        words = sorted(self.all_words(), key=lambda tup: tup[1])
        return words


class RadixNode:
    """
    Um nó da árvore de prefixos compacta.
    """

    def __init__(
        self, prefix: str = "", is_leaf: bool = False, index: int = None
    ) -> None:
        """
        Cria um nó da árvore de prefixos compacta.

        Args:
            prefix (str): prefixo do nó.
            is_leaf (bool): se o nó é uma folha.
            index (int): índice da palavra adicionada

        >>> RadixNode()
        >>> RadixNode("prefix")
        >>> RadixNode("prefix", True)
        """
        # Mapeamento a partir do primeiro caractere do prefixo do nó
        self.nodes: dict[str, RadixNode] = {}
        # Um nó será uma folha se a árvore conter sua palavra
        self.is_leaf = is_leaf
        self.prefix = prefix
        self.index = index

    def _match(self, word: str) -> tuple[str, str, str]:
        """
        Compute the common substring of the prefix of the node and a word

        Args:
            word (str): word to compare

        Returns:
            (str, str, str): common substring, remaining prefix, remaining word

        >>> RadixNode("myprefix")._match("mystring")
        ('my', 'prefix', 'string')
        """
        x = 0
        for q, w in zip(self.prefix, word):
            if q != w:
                break

            x += 1

        return self.prefix[:x], self.prefix[x:], word[x:]

    def insert(self, word: str, index: int) -> None:
        # Caso 1: Se a palavra for o prefixo do nó
        # Solução: Definimos o nó atual como folha
        if self.prefix == word and not self.is_leaf:
            self.is_leaf = True

        # Caso 2: O nó não tem arestas que tenham um prefixo para a palavra
        # Solução: Criamos uma aresta do nó atual para um novo
        # contendo a palavra
        elif word[0] not in self.nodes:
            self.nodes[word[0]] = RadixNode(word, True, index)

        else:
            incoming_node = self.nodes[word[0]]
            matching_string, remaining_prefix, remaining_word = incoming_node._match(
                word
            )

            # Caso 3: O prefixo do nó é igual ao correspondente
            # Solução: Inserimos a palavra restante no próximo nó
            if remaining_prefix == "":
                self.nodes[matching_string[0]].insert(remaining_word, index)

            # Caso 4: A palavra é maior que a correspondência
            # Solução: Crie um nó entre os dois nós, altere
            # prefixos e adicione o novo nó para a palavra restante
            else:
                incoming_node.prefix = remaining_prefix

                aux_node = self.nodes[matching_string[0]]
                self.nodes[matching_string[0]] = RadixNode(
                    matching_string, False, index
                )
                self.nodes[matching_string[0]].nodes[remaining_prefix[0]] = aux_node

                if remaining_word == "":
                    self.nodes[matching_string[0]].is_leaf = True
                else:
                    self.nodes[matching_string[0]].insert(remaining_word, index)

    def find(self, word: str) -> bool:
        incoming_node = self.nodes.get(word[0], None)
        if not incoming_node:
            return False
        else:
            matching_string, remaining_prefix, remaining_word = incoming_node._match(
                word
            )
            # Se houver prefixo restante, a palavra não pode estar na árvore
            if remaining_prefix != "":
                return False
            # Isso se aplica quando a palavra e o prefixo são iguais
            elif remaining_word == "":
                return incoming_node.is_leaf
            # Temos palavras restantes, então verificamos o próximo nó
            else:
                return incoming_node.find(remaining_word)

    def delete(self, word: str) -> bool:
        incoming_node = self.nodes.get(word[0], None)
        if not incoming_node:
            return False
        else:
            matching_string, remaining_prefix, remaining_word = incoming_node._match(
                word
            )
            # Se houver prefixo restante, a palavra não pode estar na árvore
            if remaining_prefix != "":
                return False
            # Temos palavras restantes, então verificamos o próximo nó
            elif remaining_word != "":
                return incoming_node.delete(remaining_word)
            # Se não for uma folha, não precisamos excluir
            elif not incoming_node.is_leaf:
                return False
            else:
                # Excluímos os nós se nenhuma aresta sair deles
                if len(incoming_node.nodes) == 0:
                    del self.nodes[word[0]]
                    # Nós mesclamos o nó atual com seu único filho
                    if len(self.nodes) == 1 and not self.is_leaf:
                        merging_node = next(iter(self.nodes.values()))
                        self.is_leaf = merging_node.is_leaf
                        self.prefix += merging_node.prefix
                        self.index = merging_node.index
                        self.nodes = merging_node.nodes
                # Se houver mais de uma aresta, nós apenas a marcamos como não-folha
                elif len(incoming_node.nodes) > 1:
                    incoming_node.is_leaf = False
                # Se houver 1 aresta, nós a mesclamos com seu filho
                else:
                    merging_node = next(iter(incoming_node.nodes.values()))
                    incoming_node.is_leaf = merging_node.is_leaf
                    incoming_node.prefix += merging_node.prefix
                    incoming_node.index = merging_node.index
                    incoming_node.nodes = merging_node.nodes
                return True

    def print_tree(self, height: int = 0) -> None:
        if self.prefix != "":
            print(
                "---" * height,
                (
                    f"{self.prefix}:    ({self.index})"
                    if self.is_leaf
                    else f"{self.prefix}"
                ),
            )

        for value in self.nodes.values():
            value.print_tree(height + 1)

    def all_words(self) -> list[(str, int)]:
        words = []
        if self.is_leaf:
            words.append((self.prefix, self.index))
        for value in self.nodes.values():
            sufixes = value.all_words()
            for suffix, index in sufixes:
                words.append((self.prefix + suffix, index))
        return words

In [ ]:
tree = RadixTree()
tree.insert_many(["word", "word1", "word2", "word3", "word12", "abc", "a", "bc"])
print(tree)
tree -= "a"
tree += "c"
tree += "bcd"
tree += "aaa"
tree -= "word12"
tree += "abd"
print(tree.sort())
tree.print_tree()

[('word', 0), ('word1', 1), ('word12', 4), ('word2', 2), ('word3', 3), ('a', 6), ('abc', 5), ('bc', 7)]
[('word', 0), ('word1', 1), ('word2', 2), ('word3', 3), ('abc', 5), ('bc', 7), ('c', 8), ('bcd', 9), ('aaa', 10), ('abd', 11)]
--- word:    (0)
------ 1:    (1)
------ 2:    (2)
------ 3:    (3)
--- a
------ b
--------- c:    (5)
--------- d:    (11)
------ aa:    (10)
--- bc:    (7)
------ d:    (9)
--- c:    (8)


In [10]:
def test_trie() -> bool:
    words = "banana bananas bandana band apple all beast".split()
    root = RadixTree()
    root.insert_many(words)
    words_list = root.all_words()
    print(words_list)
    assert len(words) == len(words_list)
    assert words.sort() == words_list.sort()
    assert not root.find("bandanas")
    assert not root.find("apps")
    root.delete("all")
    assert not root.find("all")
    root.delete("banana")
    assert not root.find("banana")
    assert root.find("bananas")
    root += "a"
    assert root.find("a")
    root -= "a"
    assert not root.find("a")
    assert not root.find("apps")
    assert root.find("apple")
    root.print_tree()
    return True


def pytests() -> None:
    assert test_trie()


def main() -> None:
    """
    >>> pytests()
    """
    pytests()


if __name__ == "__main__":
    main()

[('banana', 0), ('bananas', 1), ('band', 3), ('bandana', 2), ('beast', 6), ('apple', 4), ('all', 5)]
--- b
------ an
--------- anas:    (1)
--------- d:    (3)
------------ ana:    (2)
------ east:    (6)
--- apple:    (4)
